# 📘 P1.3.2.5 – Flask Basics
## Topic: Flask Forms and Request Data

## 🎯 Learning Objectives
By the end of this notebook, you will:
- Create HTML forms in Flask
- Understand the difference between GET and POST methods
- Access form data using request.form
- Access query parameters using request.args
- Validate user input
- Handle file uploads
- Redirect after processing forms

## 🎫 HTML Forms Basics

### What is a Form?
A **form** is a way for users to send data to a web server.

### Form Elements
- `<input type="text">` – Text input
- `<input type="password">` – Password input (hidden)
- `<input type="email">` – Email input
- `<textarea>` – Multi-line text
- `<select>` – Dropdown menu
- `<input type="checkbox">` – Checkbox
- `<input type="radio">` – Radio button
- `<input type="submit">` – Submit button

### Form Attributes
```html
<form method="POST" action="/submit">
  <input type="text" name="username">
  <input type="submit">
</form>
```
- `method="POST"` – Send data in body (secure)
- `method="GET"` – Send data in URL (visible)
- `action="/submit"` – Where to send the form
- `name="field"` – Identifies the field in request.form

## 📨 GET vs POST

### GET Method
- Data sent in URL: `/?name=Alice&age=25`
- Limited size (~2000 characters)
- Visible in browser history
- Use for: Searches, filters, public data

### POST Method
- Data sent in request body (hidden)
- No size limit
- Not visible in URL
- Use for: Logins, file uploads, sensitive data

### Accessing Data
```python
# GET: http://localhost:5000/?q=python
query = request.args.get('q')  # 'python'

# POST: Form with name="username"
username = request.form.get('username')
```

## 🎯 Working with Form Data

### Basic Form Example
**HTML Form:**
```html
<form method="POST" action="/login">
  <input type="text" name="username" required>
  <input type="password" name="password" required>
  <input type="submit" value="Login">
</form>
```

**Flask Handler:**
```python
@app.route('/login', methods=['GET', 'POST'])
def login():
    if request.method == 'POST':
        username = request.form.get('username')
        password = request.form.get('password')
        # Process login
        return f"Logging in: {username}"
    return render_template('login.html')
```

In [ ]:
from flask import Flask, render_template, request

app = Flask(__name__)

# Display form (GET) and process form (POST)
@app.route('/contact', methods=['GET', 'POST'])
def contact():
    if request.method == 'POST':
        name = request.form.get('name')
        email = request.form.get('email')
        message = request.form.get('message')
        return f"Thank you {name}! We received your message: {message}"
    return render_template('contact_form.html')

# Simple GET form with query parameters
@app.route('/search')
def search():
    query = request.args.get('q', 'No search term')
    return f"You searched for: {query}"

if __name__ == '__main__':
    app.run(debug=True)

**💡 To see this in action, run:** `app_example1.py` in the folder

Try:
- `http://localhost:5000/contact` → Fill and submit form
- `http://localhost:5000/search?q=python` → Query parameter search

## ✅ Input Validation

### Why Validate?
- Ensure data quality
- Prevent errors and crashes
- Security: Block malicious input

### Validation Techniques
```python
# Check if field exists
if 'username' not in request.form:
    return "Username required"

# Check length
username = request.form.get('username')
if len(username) < 3:
    return "Username must be at least 3 characters"

# Check type
try:
    age = int(request.form.get('age'))
except ValueError:
    return "Age must be a number"
```

In [ ]:
from flask import Flask, render_template, request

app = Flask(__name__)

# Registration form with validation
@app.route('/register', methods=['GET', 'POST'])
def register():
    if request.method == 'POST':
        username = request.form.get('username', '')
        email = request.form.get('email', '')
        age = request.form.get('age', '')
        
        # Validation
        if len(username) < 3:
            return "Username must be at least 3 characters"
        if '@' not in email:
            return "Invalid email"
        try:
            age_int = int(age)
            if age_int < 13:
                return "Must be 13 or older"
        except ValueError:
            return "Age must be a number"
        
        return f"Registration successful! Welcome {username}"
    return render_template('register_form.html')

if __name__ == '__main__':
    app.run(debug=True)

**💡 To see this in action, run:** `app_example2.py` in the folder

Try:
- Enter a username less than 3 characters
- Enter an invalid email
- Enter a non-numeric age
- Fill all fields correctly to register

## 📁 File Uploads

### HTML File Upload Form
```html
<form method="POST" enctype="multipart/form-data">
  <input type="file" name="file">
  <input type="submit">
</form>
```

**Important:** `enctype="multipart/form-data"` must be set for file uploads

### Processing File Uploads
```python
file = request.files.get('file')
if file:
    filename = file.filename  # Get original filename
    file.save(f'uploads/{filename}')  # Save to disk
```

In [ ]:
from flask import Flask, render_template, request
import os

app = Flask(__name__)
UPLOAD_FOLDER = 'uploads'
if not os.path.exists(UPLOAD_FOLDER):
    os.makedirs(UPLOAD_FOLDER)
app.config['UPLOAD_FOLDER'] = UPLOAD_FOLDER

# File upload handler
@app.route('/upload', methods=['GET', 'POST'])
def upload():
    if request.method == 'POST':
        if 'file' not in request.files:
            return "No file selected"
        
        file = request.files['file']
        if file.filename == '':
            return "No file selected"
        
        # Check file type
        allowed_ext = {'txt', 'pdf', 'png', 'jpg'}
        ext = file.filename.split('.')[-1].lower()
        if ext not in allowed_ext:
            return f"File type .{ext} not allowed"
        
        filename = file.filename
        file.save(os.path.join(app.config['UPLOAD_FOLDER'], filename))
        return f"File {filename} uploaded successfully!"
    
    return render_template('upload_form.html')

if __name__ == '__main__':
    app.run(debug=True)

**💡 To see this in action, run:** `app_example3.py` in the folder

Try:
- Upload a .txt or .pdf file
- Try uploading an unsupported file type
- Check the uploads/ folder for saved files

## 🔀 Redirects and Multi-Step Forms

### Redirect After Submission
```python
from flask import redirect, url_for

@app.route('/submit', methods=['POST'])
def submit():
    # Process form
    return redirect(url_for('success'))  # Redirect to /success

@app.route('/success')
def success():
    return "Form submitted successfully!"
```

### Why Redirect?
- Prevent form resubmission when user refreshes
- Show success page with clean URL
- Better user experience

In [ ]:
from flask import Flask, render_template, request, redirect, url_for

app = Flask(__name__)

# Feedback form with redirect
@app.route('/feedback', methods=['GET', 'POST'])
def feedback():
    if request.method == 'POST':
        name = request.form.get('name')
        rating = request.form.get('rating')
        comment = request.form.get('comment')
        
        # Process feedback (save to database, send email, etc.)
        # ...
        
        return redirect(url_for('thank_you', name=name))
    
    return render_template('feedback_form.html')

# Thank you page
@app.route('/thank-you')
def thank_you():
    name = request.args.get('name', 'Guest')
    return render_template('thank_you.html', name=name)

if __name__ == '__main__':
    app.run(debug=True)

**💡 To see this in action, run:** `app_example4.py` in the folder

Try:
- Fill and submit the feedback form
- Notice the redirect to thank-you page
- Try refreshing – no form resubmission

## ❌ Common Mistakes

### Mistake 1: Not Specifying POST in Route
❌ Bad: `@app.route('/login')` (defaults to GET only)
✅ Good: `@app.route('/login', methods=['GET', 'POST'])`

### Mistake 2: Using request.form for Query Parameters
❌ Bad: `query = request.form.get('q')` (when using GET)
✅ Good: `query = request.args.get('q')`

### Mistake 3: Not Checking if Field Exists
❌ Bad: `value = request.form['field']` (crashes if missing)
✅ Good: `value = request.form.get('field', 'default')`

### Mistake 4: Forgetting multipart/form-data for Files
❌ Bad: `<form method="POST">` for file upload
✅ Good: `<form method="POST" enctype="multipart/form-data">`

### Mistake 5: Not Validating User Input
❌ Bad: `int(request.form.get('age'))` (crashes on non-numeric)
✅ Good: Use try-except or check input before converting

## ✅ Key Takeaways
- HTML forms send data via GET (URL) or POST (body)
- Use `request.args.get()` for GET parameters
- Use `request.form.get()` for POST form data
- Always validate user input before using it
- Use `request.files` for file uploads
- Always set `enctype="multipart/form-data"` for file forms
- Redirect after form submission to prevent resubmission
- Use `request.method == 'POST'` to detect form submissions
- Provide meaningful error messages to users